In [2]:
import mediapipe as mp
import cv2
import numpy as np

In [3]:
from PIL import ImageFont, ImageDraw, Image
import numpy as np

def draw_text(img, text, position, font_size, font_color):
    font_path = "C:/Windows/Fonts/gulim.ttc"  # Windows에서 Gulim 폰트 경로

    # opencv 이미지를 PIL이미지로 변환
    img_pil = Image.fromarray(img)

    # PIL Draw 객체 생성
    draw = ImageDraw.Draw(img_pil)

    # 폰트 스타일 지정
    font = ImageFont.truetype(font_path, font_size)

    # PIL 이미지에 텍스트 입력
    draw.text(position, text, font=font, fill=font_color)
    return np.array(img_pil) # 최종 numpy array 로 이미지 형태 반환

In [7]:
from PIL import ImageFont, ImageDraw, Image
import numpy as np

def draw_text(img, text, position, font_size, font_color):
    font_path = "C:/Windows/Fonts/gulim.ttc"  # Windows에서 Gulim 폰트 경로

    # opencv 이미지를 PIL이미지로 변환
    img_pil = Image.fromarray(img)

    # PIL Draw 객체 생성
    draw = ImageDraw.Draw(img_pil)

    # 폰트 스타일 지정
    font = ImageFont.truetype(font_path, font_size)

    # PIL 이미지에 텍스트 입력
    draw.text(position, text, font=font, fill=font_color)
    return np.array(img_pil) # 최종 numpy array 로 이미지 형태 반환

In [8]:
# MediaPipe Pose 설정
pose = mp.solutions.pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    smooth_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 그리기 함수 초기화
mp_drawing = mp.solutions.drawing_utils
#####################################################################################################


def get_pose(pose_landmarks):

    # 왼쪽 어깨(shoulder) 11 // 오른쪽 어깨(shoulder) 12
    left_shoulder, right_shoulder = pose_landmarks.landmark[11], pose_landmarks.landmark[12]
    
    # 왼쪽 팔꿈치(elbow) 13 // 오른쪽 팔꿈치(elbow) 14
    left_elbow, right_elbow = pose_landmarks.landmark[13], pose_landmarks.landmark[14]
    
    # 왼쪽 손목(wrist) 15 // 오른쪽 손목(wrist) 16
    left_wrist, right_wrist = pose_landmarks.landmark[15], pose_landmarks.landmark[16]

    # 왼쪽 손바닥(palm) 19 // 오른쪽 손바닥(palm) 20
    left_palm, right_palm = pose_landmarks.landmark[19], pose_landmarks.landmark[20]

    # 입
    mouth = pose_landmarks.landmark[10]

    # 얼굴
    face = pose_landmarks.landmark[0]

                           # 팔꿈치.x < 어깨.x                    손목.y > 팔꿈치.y             손바닥.y > 팔꿈치.y             손바닥.x < 어깨.x                     손바닥.x > 오른어깨.x 
    b_Next_Level_Left = left_elbow.x < left_shoulder.x and left_wrist.y < left_shoulder.y and left_palm.y < left_elbow.y and left_palm.x < left_shoulder.x and left_palm.x > right_shoulder.x
    # b_Next_Level_Right = right_elbow.x > right_shoulder

    b_victory = left_wrist.y < face.y and right_wrist.y < face.y and left_wrist.x > right_wrist.x
    b_exit = left_wrist.y < left_shoulder.y and right_wrist.y < right_shoulder.y and left_wrist.x < right_wrist.x
    b_armCross = left_wrist.y < left_elbow.y and right_wrist.y < right_elbow.y and left_wrist.x < left_elbow.x and right_wrist.x > right_elbow.x and left_wrist.x < right_wrist.x
    
    # 포즈 판별
    if b_Next_Level_Left:
        return "Next Level"
    elif b_victory:
        return "만세"
    # elif b_exit:
    #     return "엑스"
    # elif b_armCross:
    #     return "팔짱"
    else:
        return "모름"
        

#####################################################################################################
# 웹캠 캡처
cap = cv2.VideoCapture(0)

while cap.isOpened():
    # 이미지 읽기
    ret, image = cap.read()
    if not ret:
        break

    # 이미지 변환
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 포즈 추정
    results = pose.process(image)

    # 추정된 포즈 정보 확인
    if results.pose_landmarks:
        # 연결된 랜드마크 시각화
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)
        
        # 감지된 포즈의 손목, 팔꿈치, 어깨의 실시간 좌표를 화면에 표시
        for landmark in [results.pose_landmarks.landmark[15],  # 왼쪽 손목
                         results.pose_landmarks.landmark[13],  # 왼쪽 팔꿈치
                         results.pose_landmarks.landmark[11],  # 왼쪽 어깨
                         results.pose_landmarks.landmark[16],  # 오른쪽 손목
                         results.pose_landmarks.landmark[14],  # 오른쪽 팔꿈치
                         results.pose_landmarks.landmark[12]]: # 오른쪽 어깨
            if landmark.visibility > 0.5:  # 랜드마크가 충분히 신뢰할 경우에만 좌표를 표시합니다
                height, width, _ = image.shape
                cx, cy = int(landmark.x * width), int(landmark.y * height)
                cv2.putText(image, f'({cx}, {cy})', (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # 세그멘테이션 마스크 추출 및 배경 제거
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # 이미지 출력
    cv2.imshow('MediaPipe Pose', image)

    # 키 입력 처리
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()

In [13]:
# MediaPipe Hands 설정
hands = mp.solutions.hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 손가락 제스처 딕셔너리
finger_gestures = {
    0: "Thumb",
    1: "Index",
    2: "Middle",
    3: "Ring",
    4: "Pinky"
}

# 손가락 제스처 판별 함수
def detect_finger_gesture(hand_landmarks):
    finger_status = [0] * 5  # 엄지, 검지, 중지, 약지, 새끼 손가락
    if hand_landmarks:
        # 손가락의 끝 부분 랜드마크 인덱스
        finger_tip_indices = [4, 8, 12, 16, 20]
        for finger, tip_index in enumerate(finger_tip_indices):
            # 해당 손가락의 끝 부분 랜드마크의 y 좌표가 이전 두 관절 랜드마크의 y 좌표보다 작으면 손가락을 오므릅니다.
            if hand_landmarks.landmark[tip_index].y < hand_landmarks.landmark[tip_index-2].y and hand_landmarks.landmark[tip_index].y < hand_landmarks.landmark[tip_index-1].y:
                finger_status[finger] = 1
    return finger_status

# MediaPipe Pose 설정
pose = mp.solutions.pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    smooth_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 그리기 함수 초기화
mp_drawing = mp.solutions.drawing_utils

# 웹캠 캡처
cap = cv2.VideoCapture(0)

while cap.isOpened():
    # 이미지 읽기
    ret, image = cap.read()
    if not ret:
        break

    # 이미지 변환
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 손 감지
    hands_results = hands.process(image)

    # 포즈 추정
    pose_results = pose.process(image)

    # 감지된 손이 있으면
    if hands_results.multi_hand_landmarks:
        # 손 감지 결과를 반복하여 처리합니다.
        for hand_landmarks in hands_results.multi_hand_landmarks:
            # 각 손가락 제스처를 판별합니다.
            finger_status = detect_finger_gesture(hand_landmarks)
            gesture_text = ""
            for idx, status in enumerate(finger_status):
                if status == 1:
                    gesture_text += finger_gestures[idx] + " "
            cv2.putText(image, gesture_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # 추정된 포즈 정보 확인
    if pose_results.pose_landmarks:
        # 연결된 랜드마크 시각화
        mp_drawing.draw_landmarks(
            image, pose_results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)
        
    # 세그멘테이션 마스크 추출 및 배경 제거
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # 이미지 출력
    cv2.imshow('MediaPipe Pose', image)

    # 키 입력 처리
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()